# 🚀 YOLOv8 Professional Training Pipeline
## CSV → YOLO Conversion · Training · Evaluation · Export · Repository Builder

**Features:**
- Auto-detect CSV annotation format (pixel boxes, width/height, normalized YOLO)
- Deterministic train/val split with configurable ratio
- Full YOLOv8 training with Ultralytics
- Evaluation, inference visualization, and model export (ONNX, TorchScript)

---
> 📌 **How to use:** Run cells sequentially. Edit the **Configuration** cell to match your dataset.

## 1 · Install Dependencies

In [ ]:
# ── Install pinned dependencies ──────────────────────────────────
!pip install -q ultralytics==8.2.103 opencv-python-headless==4.10.0.84 \
    pandas==2.2.2 pyyaml==6.0.2 matplotlib==3.9.2 onnx==1.16.2 \
    onnxruntime==1.19.2 Pillow==10.4.0 tqdm==4.66.5 seaborn==0.13.2

import ultralytics
print(f"Ultralytics version: {ultralytics.__version__}")

## 2 · Configuration

Edit the paths and hyperparameters below to match your dataset.

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CONFIGURATION — edit these values
# ══════════════════════════════════════════════════════════════════

# ── Paths ────────────────────────────────────────────────────────
IMAGES_DIR   = "/content/data/images"       # folder with jpg/png images
CSV_PATH     = "/content/data/annotations.csv"  # CSV annotation file
WORK_DIR     = "/content/work"              # intermediate files
OUTPUT_DIR   = "/content/output"            # final artifacts
REPO_DIR     = "/content/repo"              # generated repository

# ── Train / Val split ────────────────────────────────────────────
SPLIT_RATIO  = 0.8     # fraction for training (rest → validation)
SEED         = 42

# ── Training hyperparameters ─────────────────────────────────────
MODEL_SIZE   = "yolov8n.pt"   # yolov8n / yolov8s / yolov8m / yolov8l / yolov8x
IMGSZ        = 640
EPOCHS       = 50
BATCH        = 16
DEVICE       = ""             # "" = auto, "0" = GPU 0, "cpu" = CPU

# ── Options ──────────────────────────────────────────────────────
ALLOW_BACKGROUND_IMAGES = True   # keep images with no labels as background
LICENSE_TYPE = "MIT"              # "MIT" or "Apache-2.0"

# ══════════════════════════════════════════════════════════════════
print("✅ Configuration loaded.")

## 3 · Imports & Utility Functions

In [ ]:
import os, shutil, random, glob, pathlib, warnings, sys, yaml, csv, json
from pathlib import Path
from collections import Counter, OrderedDict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")
random.seed(SEED)
np.random.seed(SEED)

# ── Helper: get image dimensions ─────────────────────────────────
def get_image_size(path):
    """Return (width, height) of an image file."""
    with Image.open(path) as img:
        return img.size  # (w, h)

# ── Helper: detect CSV format ────────────────────────────────────
def detect_csv_format(df):
    """
    Auto-detect CSV annotation format.
    Returns one of: 'A' (xmin,ymin,xmax,ymax), 'B' (xmin,ymin,w,h),
                     'C' (normalized YOLO), or raises ValueError.
    """
    cols_lower = [c.lower().strip() for c in df.columns]

    # Check for 'normalized' column → Format C
    if 'normalized' in cols_lower:
        print("📐 Detected Format C (normalized YOLO-like)")
        return 'C'

    # Check column names for xmax/ymax → Format A
    if 'xmax' in cols_lower and 'ymax' in cols_lower:
        print("📐 Detected Format A (xmin, ymin, xmax, ymax)")
        return 'A'

    # Check for width/height columns → Format B
    if 'width' in cols_lower and 'height' in cols_lower:
        print("📐 Detected Format B (xmin, ymin, width, height)")
        return 'B'

    # Fallback: try positional detection
    if len(df.columns) >= 6:
        # Check if values look normalized (all between 0 and 1)
        numeric_cols = df.iloc[:, 1:5]
        try:
            numeric_vals = numeric_cols.astype(float)
            if numeric_vals.max().max() <= 1.0 and numeric_vals.min().min() >= 0.0:
                print("📐 Auto-detected Format C (values appear normalized)")
                return 'C'
        except (ValueError, TypeError):
            pass

        print("📐 Falling back to Format A (positional: col1-4 as xmin,ymin,xmax,ymax)")
        return 'A'

    raise ValueError(
        "❌ Cannot detect CSV format. Expected columns like:\n"
        "  Format A: filename, xmin, ymin, xmax, ymax, class\n"
        "  Format B: filename, xmin, ymin, width, height, class\n"
        "  Format C: filename, x_center, y_center, width, height, class, normalized"
    )

# ── Helper: normalize columns ────────────────────────────────────
def normalize_columns(df, fmt):
    """Rename columns to a standard set based on detected format."""
    df = df.copy()
    cols = list(df.columns)

    if fmt == 'A':
        mapping = {}
        cols_lower = {c.lower().strip(): c for c in cols}
        for target, candidates in [
            ('filename', ['filename', 'file', 'image', 'image_name', 'img']),
            ('xmin', ['xmin', 'x_min', 'x1', 'left']),
            ('ymin', ['ymin', 'y_min', 'y1', 'top']),
            ('xmax', ['xmax', 'x_max', 'x2', 'right']),
            ('ymax', ['ymax', 'y_max', 'y2', 'bottom']),
            ('class', ['class', 'label', 'class_name', 'category', 'cls']),
        ]:
            for c in candidates:
                if c in cols_lower:
                    mapping[cols_lower[c]] = target
                    break
        if len(mapping) < 6:
            # positional fallback
            mapping = {cols[0]: 'filename', cols[1]: 'xmin', cols[2]: 'ymin',
                       cols[3]: 'xmax', cols[4]: 'ymax', cols[5]: 'class'}
        df = df.rename(columns=mapping)

    elif fmt == 'B':
        cols_lower = {c.lower().strip(): c for c in cols}
        mapping = {}
        for target, candidates in [
            ('filename', ['filename', 'file', 'image', 'image_name', 'img']),
            ('xmin', ['xmin', 'x_min', 'x1', 'left', 'x']),
            ('ymin', ['ymin', 'y_min', 'y1', 'top', 'y']),
            ('width', ['width', 'w', 'box_width']),
            ('height', ['height', 'h', 'box_height']),
            ('class', ['class', 'label', 'class_name', 'category', 'cls']),
        ]:
            for c in candidates:
                if c in cols_lower:
                    mapping[cols_lower[c]] = target
                    break
        if len(mapping) < 6:
            mapping = {cols[0]: 'filename', cols[1]: 'xmin', cols[2]: 'ymin',
                       cols[3]: 'width', cols[4]: 'height', cols[5]: 'class'}
        df = df.rename(columns=mapping)

    elif fmt == 'C':
        cols_lower = {c.lower().strip(): c for c in cols}
        mapping = {}
        for target, candidates in [
            ('filename', ['filename', 'file', 'image', 'image_name', 'img']),
            ('x_center', ['x_center', 'cx', 'center_x', 'x']),
            ('y_center', ['y_center', 'cy', 'center_y', 'y']),
            ('width', ['width', 'w', 'box_width']),
            ('height', ['height', 'h', 'box_height']),
            ('class', ['class', 'label', 'class_name', 'category', 'cls']),
        ]:
            for c in candidates:
                if c in cols_lower:
                    mapping[cols_lower[c]] = target
                    break
        if len(mapping) < 6:
            mapping = {cols[0]: 'filename', cols[1]: 'x_center', cols[2]: 'y_center',
                       cols[3]: 'width', cols[4]: 'height', cols[5]: 'class'}
        df = df.rename(columns=mapping)

    # Normalize filename to basename
    df['filename'] = df['filename'].apply(lambda x: os.path.basename(str(x).strip()))
    return df

# ── Helper: convert to YOLO format ───────────────────────────────
def convert_to_yolo(row, fmt, img_w, img_h):
    """
    Convert a single annotation row to YOLO format:
    (x_center, y_center, width, height) all normalized to [0, 1].
    Returns None if the box is invalid.
    """
    try:
        if fmt == 'A':
            xmin = float(row['xmin'])
            ymin = float(row['ymin'])
            xmax = float(row['xmax'])
            ymax = float(row['ymax'])
        elif fmt == 'B':
            xmin = float(row['xmin'])
            ymin = float(row['ymin'])
            xmax = xmin + float(row['width'])
            ymax = ymin + float(row['height'])
        elif fmt == 'C':
            # Already normalized
            xc = float(row['x_center'])
            yc = float(row['y_center'])
            w  = float(row['width'])
            h  = float(row['height'])
            # Clamp
            xc = max(0.0, min(1.0, xc))
            yc = max(0.0, min(1.0, yc))
            w  = max(0.0, min(1.0, w))
            h  = max(0.0, min(1.0, h))
            if w <= 0 or h <= 0:
                return None
            return (xc, yc, w, h)

        # For A and B: clamp to image bounds
        xmin = max(0, min(img_w, xmin))
        ymin = max(0, min(img_h, ymin))
        xmax = max(0, min(img_w, xmax))
        ymax = max(0, min(img_h, ymax))

        bw = xmax - xmin
        bh = ymax - ymin
        if bw <= 0 or bh <= 0:
            return None

        xc = (xmin + xmax) / 2.0 / img_w
        yc = (ymin + ymax) / 2.0 / img_h
        w  = bw / img_w
        h  = bh / img_h
        return (xc, yc, w, h)

    except (ValueError, TypeError):
        return None

# ── Helper: build class mapping ──────────────────────────────────
def build_class_mapping(classes_series):
    """
    Build a mapping from class label to integer ID.
    If labels are already integers, validate contiguous from 0; remap if needed.
    Returns: dict {label: int_id}, list of class names ordered by id.
    """
    unique_classes = classes_series.unique()

    # Check if all are integers
    all_int = True
    for c in unique_classes:
        try:
            int(c)
        except (ValueError, TypeError):
            all_int = False
            break

    if all_int:
        int_classes = sorted([int(c) for c in unique_classes])
        if int_classes == list(range(len(int_classes))):
            mapping = {str(c): c for c in int_classes}
            names = [str(c) for c in int_classes]
        else:
            # Remap
            mapping = {str(c): i for i, c in enumerate(int_classes)}
            names = [str(c) for c in int_classes]
            print(f"⚠️  Class IDs are not contiguous. Remapped: {mapping}")
    else:
        sorted_classes = sorted([str(c) for c in unique_classes])
        mapping = {c: i for i, c in enumerate(sorted_classes)}
        names = sorted_classes

    return mapping, names

# ── Helper: visualize boxes ──────────────────────────────────────
def visualize_yolo_labels(image_path, label_path, class_names, ax=None):
    """Draw YOLO bounding boxes on an image."""
    img = Image.open(image_path)
    w, h = img.size
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.imshow(img)
    ax.set_title(os.path.basename(image_path), fontsize=9)
    ax.axis('off')

    colors = plt.cm.Set3(np.linspace(0, 1, max(len(class_names), 1)))

    if os.path.exists(label_path):
        with open(label_path) as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                cls_id = int(parts[0])
                xc, yc, bw, bh = [float(x) for x in parts[1:]]
                x1 = (xc - bw / 2) * w
                y1 = (yc - bh / 2) * h
                rect_w = bw * w
                rect_h = bh * h
                color = colors[cls_id % len(colors)]
                rect = patches.Rectangle((x1, y1), rect_w, rect_h,
                                         linewidth=2, edgecolor=color,
                                         facecolor='none')
                ax.add_patch(rect)
                label = class_names[cls_id] if cls_id < len(class_names) else str(cls_id)
                ax.text(x1, y1 - 4, label, fontsize=8, color='white',
                        bbox=dict(boxstyle='round,pad=0.2', facecolor=color, alpha=0.8))
    return ax

print("✅ Utility functions loaded.")

## 4 · Data Validation & Inspection

Read the CSV, auto-detect format, validate data, and display summary statistics.

In [ ]:
# ── Read CSV ─────────────────────────────────────────────────────
df = pd.read_csv(CSV_PATH)
print(f"📄 CSV loaded: {len(df)} rows, {len(df.columns)} columns")
print(f"   Columns: {list(df.columns)}")
print()
df.head(10)

In [ ]:
# ── Detect format & normalize columns ────────────────────────────
csv_format = detect_csv_format(df)
df = normalize_columns(df, csv_format)
print(f"\n📊 Format: {csv_format}")
print(f"   Unique classes: {df['class'].nunique()}")
print(f"   Class distribution:")
print(df['class'].value_counts().to_string())
print(f"\n   Unique images in CSV: {df['filename'].nunique()}")

In [ ]:
# ── Validate image files exist ───────────────────────────────────
image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff', '.webp'}
available_images = {}
for f in os.listdir(IMAGES_DIR):
    if pathlib.Path(f).suffix.lower() in image_extensions:
        available_images[f] = os.path.join(IMAGES_DIR, f)

csv_filenames = set(df['filename'].unique())
found = csv_filenames & set(available_images.keys())
missing = csv_filenames - set(available_images.keys())

print(f"🖼️  Images in folder: {len(available_images)}")
print(f"   Matched to CSV:   {len(found)}")
if missing:
    print(f"   ⚠️  Missing files:  {len(missing)}")
    for m in list(missing)[:10]:
        print(f"      - {m}")
    if len(missing) > 10:
        print(f"      ... and {len(missing) - 10} more")
    # Remove rows with missing images
    df = df[df['filename'].isin(found)]
    print(f"   → Kept {len(df)} rows after removing missing images.")

assert len(found) > 0, "❌ No matching images found. Check IMAGES_DIR and CSV filenames."
print("\n✅ Data validation passed.")

In [ ]:
# ── Validate & clean bounding boxes ──────────────────────────────
initial_count = len(df)
clamped_count = 0

if csv_format in ('A', 'B'):
    # Check for NaN in coordinate columns
    coord_cols = ['xmin', 'ymin']
    coord_cols += ['xmax', 'ymax'] if csv_format == 'A' else ['width', 'height']
    df = df.dropna(subset=coord_cols + ['class', 'filename'])

    # Convert to numeric
    for col in coord_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df = df.dropna(subset=coord_cols)

elif csv_format == 'C':
    coord_cols = ['x_center', 'y_center', 'width', 'height']
    df = df.dropna(subset=coord_cols + ['class', 'filename'])
    for col in coord_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df = df.dropna(subset=coord_cols)

dropped = initial_count - len(df)
if dropped > 0:
    print(f"⚠️  Dropped {dropped} rows with NaN/invalid values.")

print(f"✅ {len(df)} valid annotation rows remaining.")
print(f"   Covering {df['filename'].nunique()} images.")

## 5 · YOLO Dataset Conversion Pipeline

Create the YOLO directory structure, split into train/val, convert annotations, and generate `data.yaml`.

In [ ]:
# ── Build class mapping ──────────────────────────────────────────
class_mapping, class_names = build_class_mapping(df['class'])
num_classes = len(class_names)
print(f"🏷️  {num_classes} classes: {class_names}")
print(f"   Mapping: {class_mapping}")

In [ ]:
# ── Create YOLO directory structure ──────────────────────────────
DATASET_DIR = os.path.join(WORK_DIR, "dataset")
for split in ['train', 'val']:
    os.makedirs(os.path.join(DATASET_DIR, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(DATASET_DIR, 'labels', split), exist_ok=True)

# ── Split images into train / val ────────────────────────────────
all_images = sorted(df['filename'].unique())
random.seed(SEED)
random.shuffle(all_images)
split_idx = int(len(all_images) * SPLIT_RATIO)
train_images = set(all_images[:split_idx])
val_images   = set(all_images[split_idx:])

# Handle background images (images in folder but not in CSV)
if ALLOW_BACKGROUND_IMAGES:
    bg_images = set(available_images.keys()) - csv_filenames
    if bg_images:
        bg_list = sorted(bg_images)
        random.shuffle(bg_list)
        bg_split = int(len(bg_list) * SPLIT_RATIO)
        train_images |= set(bg_list[:bg_split])
        val_images   |= set(bg_list[bg_split:])
        print(f"🖼️  Added {len(bg_images)} background images (no labels).")

print(f"📂 Train: {len(train_images)} images | Val: {len(val_images)} images")

In [ ]:
# ── Convert annotations & copy images ────────────────────────────
stats = {'train': 0, 'val': 0, 'boxes': 0, 'skipped_boxes': 0}

for img_name in tqdm(sorted(train_images | val_images), desc="Converting"):
    split = 'train' if img_name in train_images else 'val'
    stats[split] += 1

    # Copy image
    src_path = available_images.get(img_name)
    if src_path is None:
        continue
    dst_img = os.path.join(DATASET_DIR, 'images', split, img_name)
    if not os.path.exists(dst_img):
        shutil.copy2(src_path, dst_img)

    # Get image dimensions
    img_w, img_h = get_image_size(src_path)

    # Get annotations for this image
    img_df = df[df['filename'] == img_name]

    # Write label file
    label_name = os.path.splitext(img_name)[0] + '.txt'
    label_path = os.path.join(DATASET_DIR, 'labels', split, label_name)

    lines = []
    for _, row in img_df.iterrows():
        result = convert_to_yolo(row, csv_format, img_w, img_h)
        if result is None:
            stats['skipped_boxes'] += 1
            continue
        xc, yc, w, h = result
        cls_id = class_mapping[str(row['class'])]
        lines.append(f"{cls_id} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}")
        stats['boxes'] += 1

    with open(label_path, 'w') as f:
        f.write("\n".join(lines))

print(f"\n✅ Conversion complete!")
print(f"   Train images: {stats['train']}")
print(f"   Val images:   {stats['val']}")
print(f"   Total boxes:  {stats['boxes']}")
if stats['skipped_boxes'] > 0:
    print(f"   ⚠️  Skipped boxes: {stats['skipped_boxes']}")

In [ ]:
# ── Generate data.yaml ───────────────────────────────────────────
data_yaml = {
    'path': DATASET_DIR,
    'train': 'images/train',
    'val': 'images/val',
    'nc': num_classes,
    'names': class_names,
}

data_yaml_path = os.path.join(DATASET_DIR, 'data.yaml')
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False, sort_keys=False)

print(f"📄 data.yaml written to: {data_yaml_path}")
print("─" * 50)
with open(data_yaml_path) as f:
    print(f.read())

## 6 · Visualization

Display sample training images with bounding boxes to verify the conversion.

In [ ]:
# ── Visualize sample training images ─────────────────────────────
train_img_dir = os.path.join(DATASET_DIR, 'images', 'train')
train_lbl_dir = os.path.join(DATASET_DIR, 'labels', 'train')

sample_images = [f for f in os.listdir(train_img_dir)
                 if pathlib.Path(f).suffix.lower() in image_extensions]

# Prefer images that have labels
labeled = [f for f in sample_images
           if os.path.getsize(os.path.join(train_lbl_dir,
              os.path.splitext(f)[0] + '.txt')) > 0
           if os.path.exists(os.path.join(train_lbl_dir,
              os.path.splitext(f)[0] + '.txt'))]

display_images = labeled[:5] if len(labeled) >= 5 else sample_images[:5]
n = len(display_images)

fig, axes = plt.subplots(1, n, figsize=(5 * n, 5))
if n == 1:
    axes = [axes]

for ax, img_name in zip(axes, display_images):
    img_path = os.path.join(train_img_dir, img_name)
    lbl_path = os.path.join(train_lbl_dir, os.path.splitext(img_name)[0] + '.txt')
    visualize_yolo_labels(img_path, lbl_path, class_names, ax=ax)

plt.suptitle("Sample Training Images with YOLO Bounding Boxes", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 7 · YOLOv8 Training

Train the model using Ultralytics. Metrics are logged automatically.

In [ ]:
from ultralytics import YOLO

# ── Initialize model ─────────────────────────────────────────────
model = YOLO(MODEL_SIZE)
print(f"🔧 Model: {MODEL_SIZE}")
print(f"   Image size: {IMGSZ}")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch: {BATCH}")
print(f"   Device: {DEVICE if DEVICE else 'auto'}")

# ── Train ────────────────────────────────────────────────────────
results = model.train(
    data=data_yaml_path,
    imgsz=IMGSZ,
    epochs=EPOCHS,
    batch=BATCH,
    device=DEVICE if DEVICE else None,
    seed=SEED,
    project=os.path.join(WORK_DIR, "runs"),
    name="train",
    exist_ok=True,
    verbose=True,
)

print("\n✅ Training complete!")

In [ ]:
# ── Display training metrics ─────────────────────────────────────
train_dir = os.path.join(WORK_DIR, "runs", "train")

# Show results plots if available
for plot_name in ['results.png', 'confusion_matrix.png', 'confusion_matrix_normalized.png']:
    plot_path = os.path.join(train_dir, plot_name)
    if os.path.exists(plot_path):
        img = Image.open(plot_path)
        fig, ax = plt.subplots(figsize=(12, 8) if 'results' in plot_name else (8, 8))
        ax.imshow(img)
        ax.axis('off')
        ax.set_title(plot_name, fontsize=14)
        plt.tight_layout()
        plt.show()

## 8 · Evaluation & Inference

Run validation and display predictions on sample images.

In [ ]:
# ── Validation ───────────────────────────────────────────────────
best_weights = os.path.join(train_dir, "weights", "best.pt")
model_best = YOLO(best_weights)

val_results = model_best.val(data=data_yaml_path, imgsz=IMGSZ, verbose=True)

print("\n📊 Validation Results:")
print(f"   mAP50:    {val_results.box.map50:.4f}")
print(f"   mAP50-95: {val_results.box.map:.4f}")
print(f"   Precision: {val_results.box.mp:.4f}")
print(f"   Recall:    {val_results.box.mr:.4f}")

In [ ]:
# ── Inference on sample validation images ────────────────────────
val_img_dir = os.path.join(DATASET_DIR, 'images', 'val')
val_samples = [f for f in os.listdir(val_img_dir)
               if pathlib.Path(f).suffix.lower() in image_extensions][:4]

if val_samples:
    fig, axes = plt.subplots(1, len(val_samples), figsize=(5 * len(val_samples), 5))
    if len(val_samples) == 1:
        axes = [axes]

    for ax, img_name in zip(axes, val_samples):
        img_path = os.path.join(val_img_dir, img_name)
        preds = model_best.predict(img_path, imgsz=IMGSZ, conf=0.25, verbose=False)
        # Plot using ultralytics built-in
        result_img = preds[0].plot()
        ax.imshow(result_img[..., ::-1])  # BGR to RGB
        ax.set_title(img_name, fontsize=9)
        ax.axis('off')

    plt.suptitle("Predictions on Validation Images", fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
else:
    print("⚠️  No validation images found for inference preview.")

## 9 · Model Export

Export the best model to ONNX and TorchScript formats, and save all artifacts.

In [ ]:
# ── Export to ONNX ───────────────────────────────────────────────
os.makedirs(OUTPUT_DIR, exist_ok=True)

try:
    onnx_path = model_best.export(format="onnx", imgsz=IMGSZ)
    print(f"✅ ONNX exported: {onnx_path}")
except Exception as e:
    print(f"⚠️  ONNX export failed: {e}")
    onnx_path = None

# ── Export to TorchScript ────────────────────────────────────────
try:
    ts_path = model_best.export(format="torchscript", imgsz=IMGSZ)
    print(f"✅ TorchScript exported: {ts_path}")
except Exception as e:
    print(f"⚠️  TorchScript export failed: {e}")
    ts_path = None

# ── Copy artifacts to output directory ───────────────────────────
shutil.copy2(best_weights, os.path.join(OUTPUT_DIR, "best.pt"))
shutil.copy2(data_yaml_path, os.path.join(OUTPUT_DIR, "data.yaml"))

if onnx_path and os.path.exists(str(onnx_path)):
    shutil.copy2(str(onnx_path), os.path.join(OUTPUT_DIR, "best.onnx"))
if ts_path and os.path.exists(str(ts_path)):
    shutil.copy2(str(ts_path), os.path.join(OUTPUT_DIR, "best.torchscript"))

# Save class names
with open(os.path.join(OUTPUT_DIR, "classes.json"), 'w') as f:
    json.dump({"names": class_names, "mapping": class_mapping}, f, indent=2)

# Save training config
train_config = {
    "model": MODEL_SIZE, "imgsz": IMGSZ, "epochs": EPOCHS,
    "batch": BATCH, "seed": SEED, "split_ratio": SPLIT_RATIO,
    "num_classes": num_classes, "class_names": class_names,
}
with open(os.path.join(OUTPUT_DIR, "train_config.json"), 'w') as f:
    json.dump(train_config, f, indent=2)

print(f"\n📦 Artifacts saved to: {OUTPUT_DIR}")
for f in sorted(os.listdir(OUTPUT_DIR)):
    size = os.path.getsize(os.path.join(OUTPUT_DIR, f))
    print(f"   {f:30s} {size / 1024:.1f} KB")

## 10 · Package & Download

Zip the repository and artifacts for download.

In [ ]:
# ── Zip the repository ───────────────────────────────────────────
import zipfile

def zip_directory(source_dir, output_path):
    """Create a zip file from a directory."""
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(source_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, os.path.dirname(source_dir))
                zipf.write(file_path, arcname)
    size_mb = os.path.getsize(output_path) / (1024 * 1024)
    print(f"   📦 {output_path} ({size_mb:.2f} MB)")

print("Packaging repository and artifacts...\n")

# Zip repository
repo_zip = "/content/yolov8_pipeline_repo.zip"
zip_directory(REPO_DIR, repo_zip)

# Zip artifacts only
artifacts_zip = "/content/yolov8_trained_artifacts.zip"
zip_directory(OUTPUT_DIR, artifacts_zip)

print(f"\n✅ Ready for download!")
print(f"   📥 Repository:  {repo_zip}")
print(f"   📥 Artifacts:   {artifacts_zip}")

# Download in Colab
try:
    from google.colab import files
    print("\n⬇️  Downloading repository zip...")
    files.download(repo_zip)
    print("⬇️  Downloading artifacts zip...")
    files.download(artifacts_zip)
except ImportError:
    print("\n💡 Not running in Colab. Files are saved at the paths above.")

## 12 · Summary

### What was created:

| Component | Description |
|-----------|-------------|
| **YOLO Dataset** | Images + labels in train/val splits with `data.yaml` |
| **Trained Model** | `best.pt` weights with logged metrics |
| **Exports** | ONNX and TorchScript formats |
| **Documentation** | README, dataset format guide, usage guide, model card |
| **Scripts** | Standalone conversion, training, and prediction scripts |
| **Package** | Installable Python package with reusable functions |
| **Configs** | YAML configuration for reproducible experiments |

### Next Steps:

1. **Fine-tune**: Adjust hyperparameters in `configs/default.yaml` and retrain
2. **Deploy**: Use the ONNX export for production inference

---
> 🎉 **Pipeline complete!** Your professional YOLOv8 repository is ready.